In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
df_initial = pd.read_csv('./data.csv',encoding='latin1',dtype={'CustomerID': str,'InvoiceID': str})
df_initial['InvoiceDate'] = pd.to_datetime(df_initial['InvoiceDate'])
print('Dataframe dimensions:', df_initial.shape)

tab_info=pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.rename(index={0:'null values (%)'}))
print(tab_info)

In [ ]:
df_initial.head()

In [ ]:
# Remove entries for customer ID NULL
df_initial.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('Dataframe dimensions:', df_initial.shape)

tab_info=pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.rename(index={0:'null values (%)'}))
print(tab_info)

In [ ]:
df_initial.drop_duplicates(inplace = True)

In [ ]:
print('Dataframe dimensions:', df_initial.shape)

In [ ]:
country_to_index = {}
count=0
countries = df_initial["Country"].to_numpy()
for i in range(countries.shape[0]):
    if countries[i] not in country_to_index:
        country_to_index[countries[i]] = count
        count+=1
print(count)
print(country_to_index)

In [ ]:
df_initial = df_initial.drop("Country", 1)

In [ ]:
df_initial.head()

In [ ]:
for i in range(countries.shape[0]):
    countries[i] = country_to_index[countries[i]]
df_initial["CountryID"] = countries
df_initial.tail()

In [ ]:
df_phase2 = df_initial
df_phase2

In [ ]:
pd.DataFrame([{'products': len(df_initial['StockCode'].value_counts()),    
               'transactions': len(df_initial['InvoiceNo'].value_counts()),
               'customers': len(df_initial['CustomerID'].value_counts()),  
              }], columns = ['products', 'transactions', 'customers'], index = ['quantity'])

In [ ]:
# Grouping Rows if they belong to same invoice no and customer id
temp = df_phase2.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
nb_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})
nb_products_per_basket[:10].sort_values('CustomerID')

In [ ]:
df_cleaned = df_phase2.copy(deep = True)
df_cleaned['QuantityCanceled'] = 0

entry_to_remove = [] 
doubtfull_entry = []

for index, col in  df_phase2.iterrows():
    if (col['Quantity'] > 0) or col['Description'] == 'Discount': continue        
    df_test = df_phase2[(df_phase2['CustomerID'] == col['CustomerID']) &
                         (df_phase2['StockCode']  == col['StockCode']) & 
                         (df_phase2['InvoiceDate'] < col['InvoiceDate']) & 
                         (df_phase2['Quantity']   > 0)].copy()

    # Cancelation WITHOUT counterpart
    if (df_test.shape[0] == 0): 
        doubtfull_entry.append(index)

    # Cancelation WITH a counterpart
    elif (df_test.shape[0] == 1): 
        index_order = df_test.index[0]
        df_cleaned.loc[index_order, 'QuantityCanceled'] = -col['Quantity']
        entry_to_remove.append(index)        

    # Various counterparts exist in orders: we delete the last one
    elif (df_test.shape[0] > 1): 
        df_test.sort_index(axis=0 ,ascending=False, inplace = True)        
        for ind, val in df_test.iterrows():
            if val['Quantity'] < -col['Quantity']: continue
            df_cleaned.loc[ind, 'QuantityCanceled'] = -col['Quantity']
            entry_to_remove.append(index) 
            break        

In [ ]:
print(len(doubtfull_entry))

In [ ]:
df_cleaned.drop(entry_to_remove,axis=0,inplace=True)
df_cleaned.drop(doubtfull_entry,axis=0,inplace=True)

In [ ]:
print(df_cleaned.shape)

In [ ]:
df_phase3 = df_cleaned

In [ ]:
df_phase3["TotalPrice"] = df_phase3["UnitPrice"]*(df_phase3["Quantity"] - df_phase3["QuantityCanceled"])
df_phase3

In [ ]:
temp = df_phase3.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['TotalPrice'].sum()
basket_price = temp.rename(columns = {'TotalPrice':'Basket Price'})
basket_price = basket_price[basket_price["Basket Price"]>0]
basket_price.head()

In [ ]:
df_phase4 = pd.read_csv('basket_price.csv', encoding = 'latin1')
df_phase4

In [ ]:
df_phase3.to_csv("df_phase3.csv")

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
lem = WordNetLemmatizer()

In [ ]:
root_freq = {}
stock_to_roots = {}
root_to_index = {}
count = 0

def is_noun(pos):
    if pos[:2] == 'NN':
        return True
    return False


for i, row in df_phase3.iterrows():
    if pd.isnull(desc):
        continue 
    desc = row["Description"]
    desc = desc.lower()
    stock = row["StockCode"]
    tokens = nltk.word_tokenize(desc)
    nouns = [word for (word, pos) in nltk.pos_tag(tokens) if is_noun(pos)] 
    vis = {}
    temp = []
    for noun in nouns:
        noun = noun.lower()
        root = lem.lemmatize(noun)
        if root not in root_freq:
            root_freq[root] = 1
            vis[root] = 1
            temp.append(root)
            root_to_index[root] = count 
            count += 1
        elif root not in vis:
            root_freq[root] += 1
            vis[root] = 1
            temp.append(root)
    stock_to_roots[stock] = temp

print(root_freq)

In [ ]:
threshold = 1400
print(len(root_freq))
top_roots = {key:val for key, val in root_freq.items() if val >= threshold}
print(len(top_roots))

In [ ]:
c = 0
pop_root_to_index = {}
for key, val in top_roots.items():
    pop_root_to_index[key] = c 
    c += 1
print(pop_root_to_index)

In [ ]:
list_products = df_phase3["StockCode"].unique()

In [ ]:
products_np = []
mean_prices = []
count_cat = [0 for _ in range(6)]
for i,stock in enumerate(list_products):
    desc = df_phase3[df_phase3["StockCode"] == stock]["Description"].iloc[0]
    temp = [stock, desc]
    roots = stock_to_roots[stock]
    t = [0 for i in range(len(top_roots))]
    for root in roots:
        if root in top_roots:
            index = pop_root_to_index[root]
            t[index] = 1
    temp.extend(t)
    mean_price = df_phase3[df_phase3["StockCode"] == stock]["UnitPrice"].mean()
    prod_cat = 0

    if mean_price < 1:
        prod_cat = 0
    elif mean_price < 2:
        prod_cat = 1
    elif mean_price < 3:
        prod_cat = 2
    elif mean_price < 5:
        prod_cat = 3
    elif mean_price < 10:
        prod_cat = 4
    else:
        prod_cat = 5

    cat_list = [0 for _ in range(6)]
    cat_list[prod_cat] = 1
    temp.extend(cat_list)
    mean_prices.append(mean_price)
    count_cat[prod_cat]+=1
    temp = np.array(temp)
    # print(temp.shape)
    products_np.append(temp)

In [ ]:
bins = np.arange(-100, 100, 1) # fixed bin size
print(min(mean_prices))
plt.xlim([min(mean_prices)-5, 20])

plt.hist(mean_prices, bins=bins, alpha=0.5)
plt.show()



In [ ]:
count_cat = np.array(count_cat)
for i in range(6):
    print(count_cat[i]/np.sum(count_cat) * 100, "%")


In [ ]:
products_np = np.array(products_np)
print(products_np.shape)

In [ ]:
cols = ["StockCode","Description"]
for i in range(185):
    cols.append("root-"+str(i+1))
for i in range(6):
    cols.append("priceRange-"+str(i+1))
df_products = pd.DataFrame(products_np,columns = cols)
df_products.to_csv("df_products.csv")

In [ ]:
df_inp = df_products.drop(["StockCode","Description",df_products.columns[0]],axis=1)
prod_np = df_inp.to_numpy().astype('int64')


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(init='k-means++', n_clusters = 5, n_init=1000)
kmeans.fit(prod_np)
clusters = kmeans.predict(prod_np)

print(clusters)

In [ ]:
print(pd.Series(clusters).value_counts())

In [ ]:
print(df_phase3.shape,len(clusters))

In [ ]:
df_phase4 = df_products[["StockCode", "Description"]]
df_phase4["Category"] = clusters
print(df_phase4 )

In [ ]:
df_phase4.to_csv("final_prod_cat.csv")

In [ ]:
stock_to_cat = {}

for i,row in df_phase4.iterrows():
    stock_to_cat[row["StockCode"]] = row["Category"]

print(stock_to_cat)

In [ ]:
stock_list = df_phase3["StockCode"].to_numpy()
cat_list = []
for i in range(stock_list.shape[0]):
    cat_list.append(stock_to_cat[stock_list[i]])

df_phase3["prod_cat"] = cat_list



In [ ]:
df_phase3_np = df_phase3.to_numpy()

In [ ]:
print(df_phase3)

In [ ]:
df_phase5 = []
for i in range(df_phase3_np.shape[0]):
    temp = [0,0,0,0,0]
    cat = df_phase3_np[i][-1]
    temp[cat] = df_phase3_np[i][-2]
    res = list(df_phase3_np[i]) + temp
    df_phase5.append(np.array(res))

df_phase5_np = np.array(df_phase5)
print(df_phase5_np)



In [ ]:
cols = list(df_phase3.columns)
cols.extend(["cat0","cat1","cat2","cat3","cat4"])
df_phase6 = pd.DataFrame(df_phase5_np,columns=cols)
print(df_phase6)

In [ ]:
df_phase6.to_csv("df_phase6.csv")

In [ ]:
invoice_to_cats = {}
for i, row in df_phase6.iterrows():
    invoice = row["InvoiceNo"]
    if invoice in invoice_to_cats:
        invoice_to_cats[invoice][3] += row["cat0"]
        invoice_to_cats[invoice][4] += row["cat1"]
        invoice_to_cats[invoice][5] += row["cat2"]
        invoice_to_cats[invoice][6] += row["cat3"]
        invoice_to_cats[invoice][7] += row["cat4"]
        invoice_to_cats[invoice][8] += row["TotalPrice"]
    else:
        invoice_to_cats[invoice]= list([row["CustomerID"],row["InvoiceNo"], row["CountryID"], row["cat0"],row["cat1"],row["cat2"],row["cat3"],row["cat4"],row["TotalPrice"] ])

print(invoice_to_cats)

In [ ]:
df_phase7 = []
for key, val in invoice_to_cats.items():
    df_phase7.append(val)

print(df_phase7)

In [ ]:
cust_to_invoice = {}

for i, row in enumerate(df_phase7):
    cust = row[0]
    if cust in cust_to_invoice:
        cust_to_invoice[cust][2] += row[3]
        cust_to_invoice[cust][3] += row[4]
        cust_to_invoice[cust][4] += row[5]
        cust_to_invoice[cust][5] += row[6]
        cust_to_invoice[cust][6] += row[7]
        cust_to_invoice[cust][7] += row[8]
        cust_to_invoice[cust][8] = min(cust_to_invoice[cust][8], row[8])
        cust_to_invoice[cust][9] = max(cust_to_invoice[cust][9], row[8])
        cust_to_invoice[cust][10] += 1
    else:
        cust_to_invoice[cust]= list([row[0], row[2], row[3],row[4],row[5],row[6],row[7],row[8],row[8],row[8], 1])

print(cust_to_invoice)

In [ ]:
df_phase8 = []
for key, val in cust_to_invoice.items():
    val.append(val[7] / val[-1])
    for i in range(2,7):
        if val[7]!=0:
            val[i] /= val[7]
        val[i] *= 100
    df_phase8.append(val)

print(df_phase8)

# Original Order: Customer ID, Country ID, Cat-0, Cat-1, Cat-2, Cat-3, Cat-4, Total, Min, Max, Mean, Frequency

In [ ]:
cols = ["CustomerID", "CountryID", "Category-0", "Category-1", "Category-2", "Category-3", "Category-4", "Total Expenditure", "Min Expenditure", "Max Expenditure", "No. of Invoices", "Mean Expenditure"]
df_phase9 = pd.DataFrame(df_phase8, columns = cols)
print(df_phase9)

In [145]:
df_phase9.to_csv("df_phase9.csv")

In [150]:

kmeans = KMeans(init='k-means++', n_clusters = 11, n_init=10000)
# df_phase9.drop("CustomerID",axis=1,inplace=True)
kmeans.fit(df_phase9)
clusters = kmeans.predict(df_phase9)


In [151]:
print(pd.Series(clusters).value_counts())

0     1277
7     1213
3     1124
8      453
5      184
10      53
2       17
6       12
1        3
4        2
9        1
dtype: int64


In [152]:
kmeans = KMeans(init='k-means++', n_clusters = 6, n_init=10000)
# df_phase9.drop("CustomerID",axis=1,inplace=True)
kmeans.fit(df_phase9)
clusters = kmeans.predict(df_phase9)

In [153]:
print(pd.Series(clusters).value_counts())

5    2053
0    2003
4     248
3      28
2       4
1       3
dtype: int64


In [155]:
kmeans = KMeans(init='k-means++', n_clusters = 7, n_init=10000)
# df_phase9.drop("CustomerID",axis=1,inplace=True)
kmeans.fit(df_phase9)
clusters = kmeans.predict(df_phase9)
print(pd.Series(clusters).value_counts())

3    2029
0    1978
6     284
4      30
2      12
1       4
5       2
dtype: int64
